# Polars
https://docs.pola.rs/user-guide/getting-started/

In [251]:
import polars as pl
import pandas as pd
import numpy as np
import datetime as dt

## Basics

In [252]:
df = pl.DataFrame(
    {
        "name": ["Alice Archer", "Ben Brown", "Chloe Cooper", "Daniel Donovan"],
        "birthdate": [
            dt.date(1997, 1, 10),
            dt.date(1985, 2, 15),
            dt.date(1983, 3, 22),
            dt.date(1981, 4, 30),
        ],
        "weight": [57.9, 72.5, 53.6, 83.1],  # (kg)
        "height": [1.56, 1.77, 1.65, 1.75],  # (m)
    }
)
# IO: https://docs.pola.rs/user-guide/io/
df.write_csv("people.csv")
df = pl.read_csv("people.csv")
df.write_parquet("people.parquet")
df = pl.read_parquet("people.parquet")
df = df.to_pandas()
df = pl.from_pandas(df)
# convert date to datetime
df = df.with_columns(pl.col("birthdate").cast(pl.Date, strict=False))
# add and drop columns
df = df.with_columns(
    [
        (pl.col("weight") / (pl.col("height") ** 2)).alias("BMI"),
        pl.col("birthdate").dt.year().alias("birth_year"),
    ]
)
df = df.drop("birth_year", "BMI")

df

name,birthdate,weight,height
str,date,f64,f64
"""Alice Archer""",1997-01-10,57.9,1.56
"""Ben Brown""",1985-02-15,72.5,1.77
"""Chloe Cooper""",1983-03-22,53.6,1.65
"""Daniel Donovan""",1981-04-30,83.1,1.75


In [253]:
# Select columns and calculate new columns
result = df.select(
    pl.col("name"),
    pl.col("birthdate").dt.year().alias("birth_year"),
    (pl.col("weight") / (pl.col("height") ** 2)).alias("bmi"),
    (pl.col("weight", "height") * 0.95).round(2).name.suffix("-5%"),
    )
result

name,birth_year,bmi,weight-5%,height-5%
str,i32,f64,f64,f64
"""Alice Archer""",1997,23.791913,55.01,1.48
"""Ben Brown""",1985,23.141498,68.88,1.68
"""Chloe Cooper""",1983,19.687787,50.92,1.57
"""Daniel Donovan""",1981,27.134694,78.94,1.66


In [254]:
# Add new columns to the DataFrame
result = df.with_columns(
    pl.col("birthdate").dt.year().alias("birth_year"),
    (pl.col("weight") / (pl.col("height") ** 2)).alias("bmi"),
    (pl.col("weight", "height") * 0.95).round(2).name.suffix("-5%"),
)
result

name,birthdate,weight,height,birth_year,bmi,weight-5%,height-5%
str,date,f64,f64,i32,f64,f64,f64
"""Alice Archer""",1997-01-10,57.9,1.56,1997,23.791913,55.01,1.48
"""Ben Brown""",1985-02-15,72.5,1.77,1985,23.141498,68.88,1.68
"""Chloe Cooper""",1983-03-22,53.6,1.65,1983,19.687787,50.92,1.57
"""Daniel Donovan""",1981-04-30,83.1,1.75,1981,27.134694,78.94,1.66


In [255]:
# Filter rows based on conditions
# result = df.filter(
#     (pl.col("weight") > 70) & (pl.col("height") > 1.7)
# )
result = df.filter(
    pl.col("weight") > 70,
    pl.col("height") > 1.7,
)
result

name,birthdate,weight,height
str,date,f64,f64
"""Ben Brown""",1985-02-15,72.5,1.77
"""Daniel Donovan""",1981-04-30,83.1,1.75


In [256]:
# Group by and aggregate data
result = df.group_by(
    (pl.col("birthdate").dt.year() // 10 * 10).alias("decade"),
    maintain_order=True,
).agg(
    pl.len().alias("sample_size"),
    pl.col("weight").mean().round(2).alias("avg_weight"),
    pl.col("height").max().alias("tallest"),
)
print(result)

shape: (2, 4)
┌────────┬─────────────┬────────────┬─────────┐
│ decade ┆ sample_size ┆ avg_weight ┆ tallest │
│ ---    ┆ ---         ┆ ---        ┆ ---     │
│ i32    ┆ u32         ┆ f64        ┆ f64     │
╞════════╪═════════════╪════════════╪═════════╡
│ 1990   ┆ 1           ┆ 57.9       ┆ 1.56    │
│ 1980   ┆ 3           ┆ 69.73      ┆ 1.77    │
└────────┴─────────────┴────────────┴─────────┘


In [257]:
# Group by and apply calculation to each group
def some_function(group: pl.DataFrame) -> pl.DataFrame:
    print(group)
    return pl.DataFrame(
        {
            "decade": group["decade"],
            "bmi_mean": (group["weight"] / (group["height"] ** 2)).mean(),
        }
    )
result = df.with_columns(
    (pl.col("birthdate").dt.year() // 10 * 10).alias("decade"),
).group_by(
    "decade",
    maintain_order=True,
).map_groups(some_function)
result

shape: (1, 5)
┌──────────────┬────────────┬────────┬────────┬────────┐
│ name         ┆ birthdate  ┆ weight ┆ height ┆ decade │
│ ---          ┆ ---        ┆ ---    ┆ ---    ┆ ---    │
│ str          ┆ date       ┆ f64    ┆ f64    ┆ i32    │
╞══════════════╪════════════╪════════╪════════╪════════╡
│ Alice Archer ┆ 1997-01-10 ┆ 57.9   ┆ 1.56   ┆ 1990   │
└──────────────┴────────────┴────────┴────────┴────────┘
shape: (3, 5)
┌────────────────┬────────────┬────────┬────────┬────────┐
│ name           ┆ birthdate  ┆ weight ┆ height ┆ decade │
│ ---            ┆ ---        ┆ ---    ┆ ---    ┆ ---    │
│ str            ┆ date       ┆ f64    ┆ f64    ┆ i32    │
╞════════════════╪════════════╪════════╪════════╪════════╡
│ Ben Brown      ┆ 1985-02-15 ┆ 72.5   ┆ 1.77   ┆ 1980   │
│ Chloe Cooper   ┆ 1983-03-22 ┆ 53.6   ┆ 1.65   ┆ 1980   │
│ Daniel Donovan ┆ 1981-04-30 ┆ 83.1   ┆ 1.75   ┆ 1980   │
└────────────────┴────────────┴────────┴────────┴────────┘


decade,bmi_mean
i32,f64
1990,23.791913
1980,23.321326
1980,23.321326
1980,23.321326


In [258]:
# Combine multiple DataFrames
df2 = pl.DataFrame(
    {
        "name": ["Ben Brown", "Daniel Donovan", "Alice Archer", "Chloe Cooper"],
        "parent": [True, False, False, False],
        "siblings": [1, 2, 3, 4],
    }
)
result = df.join(df2, on="name", how="left")
result

name,birthdate,weight,height,parent,siblings
str,date,f64,f64,bool,i64
"""Alice Archer""",1997-01-10,57.9,1.56,false,3
"""Ben Brown""",1985-02-15,72.5,1.77,true,1
"""Chloe Cooper""",1983-03-22,53.6,1.65,false,4
"""Daniel Donovan""",1981-04-30,83.1,1.75,false,2


In [259]:
# Concatenate DataFrames
df3 = pl.DataFrame(
    {
        "name": ["Ethan Edwards", "Fiona Foster", "Grace Gibson", "Henry Harris"],
        "birthdate": [
            dt.date(1977, 5, 10),
            dt.date(1975, 6, 23),
            dt.date(1973, 7, 22),
            dt.date(1971, 8, 3),
        ],
        "weight": [67.9, 72.5, 57.6, 93.1],  # (kg)
        "height": [1.76, 1.6, 1.66, 1.8],  # (m)
    }
)

result = pl.concat([df, df3], how="vertical")
result

name,birthdate,weight,height
str,date,f64,f64
"""Alice Archer""",1997-01-10,57.9,1.56
"""Ben Brown""",1985-02-15,72.5,1.77
"""Chloe Cooper""",1983-03-22,53.6,1.65
"""Daniel Donovan""",1981-04-30,83.1,1.75
"""Ethan Edwards""",1977-05-10,67.9,1.76
"""Fiona Foster""",1975-06-23,72.5,1.6
"""Grace Gibson""",1973-07-22,57.6,1.66
"""Henry Harris""",1971-08-03,93.1,1.8
